In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 5
%autosave 15

from __future__ import division
import operator
import functools
import numpy as np
import scipy as sp
import pandas as pd
import math
import random
import pylab as pl
import matplotlib.pyplot as plt
import sklearn
import os
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from collections import defaultdict
from math import log
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import KFold
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import mean_squared_error
import base64
import csv
import gzip
import zlib

from collections import namedtuple

Autosaving every 15 seconds


C:\Users\Daria\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
TRACE_NUM = 1000
import logging
from imp import reload
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

def trace(items_num, trace_num=TRACE_NUM):
    if items_num % trace_num == 0: logging.info("Complete items %05d" % items_num)

In [3]:
def to_utf8(text):
    if isinstance(text, str): text = text.encode('utf8', "ignore")
    return text

def convert2unicode(f):
    def tmp(text):
        if not isinstance(text, str): text = text.decode('utf8', "ignore")
        return f(text)
    return tmp

def convert2lower(f):
    def tmp(text):        
        return f(text.lower())
    return tmp

#P.S. Декораторы могут усложнять отладку, так что от них вполне можно отказаться и воспользоваться copy-paste

In [4]:
from html.parser import HTMLParser
import re

class TextHTMLParser(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self._text = []
        self._title = []
        self._in_title = False
        self._in_a = False
        self._a = []

    def handle_data(self, data): 
        text = data.strip()        
        if len(text) > 0:
            text = re.sub('[ \t\r\n]+', ' ', text)
            if (self._in_title == True):
                self._title.append(text + ' ')
                self._in_title = False
            else:
                self._text.append(text + ' ')
                if (self._in_a == True):
                    self._a.append(text + ' ')

    def handle_starttag(self, tag, attrs):        
        if tag == 'p':
            self._text.append('\n\n')
        elif tag == 'br':
            self._text.append('\n')
        elif tag == 'title':
            self._in_title = True
        elif tag == 'a':
            self._in_a = True

    def handle_startendtag(self, tag, attrs):        
        if tag == 'br':
            self._text.append('\n\n')
        elif tag == 'title':
            self._in_title = False
            print('/TITLE')
        elif tag == 'a':
            self._in_a = False

    def text(self):
        return (''.join(self._title).strip(), ''.join(self._text).strip(), ''.join(self._a).strip())

@convert2unicode
def html2text_parser(text):
    parser = TextHTMLParser()
    parser.feed(text)
    return parser.text()

In [5]:
@convert2lower
@convert2unicode
def easy_tokenizer(text):
    word = str()
    for symbol in text:
        if symbol.isalnum(): word += symbol
        elif word:
            yield word
            word = str()
    if word: yield word

PYMORPHY_CACHE = {}
MORPH = None
#hint, чтобы установка pymorphy2 не была бы обязательной
def get_lemmatizer():
    import pymorphy2
    global MORPH
    if MORPH is None: MORPH = pymorphy2.MorphAnalyzer()
    return MORPH

@convert2lower
@convert2unicode
def pymorphy_tokenizer(text):
    global PYMORPHY_CACHE
    for word in easy_tokenizer(text):
        word_hash = hash(word)
        if word_hash not in PYMORPHY_CACHE:
            PYMORPHY_CACHE[word_hash] = get_lemmatizer().parse(word)[0].normal_form            
        yield PYMORPHY_CACHE[word_hash]

In [6]:
# return [title, text, text in <a>]
def html2word(raw_html, to_text=html2text_parser, tokenizer=easy_tokenizer):
    tt = to_text(raw_html)
    doc = []
    for t in tt:
        doc.append(list(tokenizer(t.lower())))
    return doc

In [7]:
DocItem = namedtuple('DocItem', ['doc_id', 'is_spam', 'url', 'features'])
SPAM = 1
LEGIT = 0
TEST_COUNT = 0
def load_csv(input_file_name):
    global TEST_COUNT
    TEST_COUNT = 0
    frequencies = defaultdict(lambda:0)
    classes = defaultdict(lambda:0)
    counts = defaultdict(lambda:0)
    with gzip.open(input_file_name) if input_file_name.endswith('gz') else open(input_file_name) as input_file:            
        headers = input_file.readline()        
        for i, line in enumerate(input_file):            
            trace(i)
            parts = line.decode('utf-8').strip().split('\t')
            url_id = int(parts[0])
            mark = int(parts[1])
            url = parts[2]
            pageInb64 = parts[3]
            html_data = base64.b64decode(pageInb64)            
            title, data, aa = html2word(html_data)     
            add_data(mark, data, frequencies, classes, counts)
#             features = calc_features_f(url, (title, data, aa, len(line)))            
#             yield DocItem(url_id, mark, url, features)
            TEST_COUNT += 1
    return frequencies, classes, counts

In [8]:
# 1	Количество слов на странице
# 2	Средняя длина слова
# 3	Количество слов в заголовке страниц (слова в теге <html><head><title> Some text </title>)
# 4	Количество слов в анкорах ссылок (<html><body><a> Some text </a>)
# 5	Коэффициент сжатия

def calc_features(url, data):
    title, words, aa, cnt_letters = data
    words_num = len(words)
    ss = sum(len(w) for w in words)
    avg_word_len = ss // words_num    
    title_words_num = len(title)
    anchor_words_num = len(aa)
    compression_level = float(cnt_letters) / ss
    
    return [words_num, avg_word_len, title_words_num, anchor_words_num, compression_level]

In [13]:
def add_data(clazz, data, frequencies, classes, counts):
    for word in data:
        frequencies[clazz, word] += 1.0
        counts[word] += 1.0
    classes[clazz] += len(data)
    
def training(frequencies, classes, counts):
    for clazz, word in frequencies:
        frequencies[clazz, word] = (frequencies[clazz, word] + 1) / (counts[word] + 1)
        # можно размытие по лапласу, как будто каждо слово видели на 1 раз больше
#         frequencies[clazz, word] = (frequencies[clazz, word] + 1.0) / (1.0 + counts[word])
    for clazz in classes:
        classes[clazz] /= TEST_COUNT
    return classes, frequencies

In [14]:
errorWeight = defaultdict(lambda:1)
errorWeight[SPAM] = 1
errorWeight[LEGIT] = 1
def classify(classifier, data):
    classes, frequencies = classifier
    #P(C|o_1..o_n) = P(c)*prod(P(o_i|c)
    # result class = argmax(c, P(c)*prod(P(o_i|c))) = argmin(c, -log(P(c) - sum(log(P(o_i|c)))))
    spamProb = -log(classes[SPAM])
    legitProb = -log(classes[LEGIT])
    for word in data:
        spamProb -= log(frequencies[SPAM, word] + 10 ** (-7))
        legitProb -= log(frequencies[LEGIT, word] + 10 ** (-7))
    spamProb -= log(errorWeight[SPAM])
    legitProb -= log(errorWeight[LEGIT])
    if (abs(spamProb - legitProb) < 20):
        return LEGIT
    if (spamProb < legitProb):
        return SPAM
    return LEGIT
#     return min(classes.keys(), key = lambda clazz: -log(classes[clazz]) -
#                sum(log(frequencies[clazz, word] + 10 ** (-7)) for word in file[1]))

In [15]:
def get_prediction(input_file_name, curClassifier):
    predict = []
    answer = []
    counts = [[0, 0], [0, 0]]
    cnt = 0    

    with gzip.open(input_file_name) if input_file_name.endswith('gz') else open(input_file_name) as input_file:            
        headers = input_file.readline()        
        for i, line in enumerate(input_file):
            trace(i)
            parts = line.decode('utf-8').strip().split('\t')
            url_id = int(parts[0])
            clazz = bool(int(parts[1]))
            url = parts[2]
            pageInb64 = parts[3]
            html_data = base64.b64decode(pageInb64)            
            title, data, aa = html2word(html_data)                        
            predClass = classify(curClassifier, data)
            predict.append(int(predClass == SPAM))
            answer.append(int(clazz == SPAM))
            counts[predClass == SPAM][clazz == SPAM] += 1
            cnt += 1
    return (counts, predict, answer)

In [16]:
# folds = [getFiles(i) for i in range(1, 11)]
# meanClassifier = defaultdict(lambda:0), defaultdict(lambda:0)
# for trains in folds:
#     curClassifier = training(trains)
#     meanClassifier = getMeanClassifier(curClassifier, meanClassifier)

TRAIN_DATA_FILE  = 'kaggle_train_data_tab.csv.gz'
# train_docs = list(load_csv(TRAIN_DATA_FILE))
frequencies, classes, counts = load_csv(TRAIN_DATA_FILE)
curClassifier = training(frequencies, classes, counts)
# kf = KFold(n_splits=2)
# for train, test in kf.split(X):
counts, predict, answer = get_prediction(TRAIN_DATA_FILE, curClassifier)
print("mean squared error = ", mean_squared_error(predict, answer))
print("f1 score = ", f1_score(predict, answer))
print("test count = ", TEST_COUNT)
print("SPAM is SPAM = ", counts[0][0])
print("SPAM is LEGIT = ", counts[0][1])
print("LEGIT is SPAM = ", counts[1][0])
print("LEGIT is LEGIT = ", counts[1][1])

01:37:08 INFO:Complete items 00000
01:37:33 INFO:Complete items 01000
01:37:58 INFO:Complete items 02000
01:38:20 INFO:Complete items 03000
01:38:42 INFO:Complete items 04000
01:39:06 INFO:Complete items 05000
01:39:26 INFO:Complete items 06000
01:39:50 INFO:Complete items 07000
01:39:52 INFO:Complete items 00000
01:40:19 INFO:Complete items 01000
01:40:41 INFO:Complete items 02000
01:41:03 INFO:Complete items 03000
01:41:26 INFO:Complete items 04000
01:41:50 INFO:Complete items 05000
01:42:11 INFO:Complete items 06000
01:42:36 INFO:Complete items 07000


mean squared error =  0.0170357751278
f1 score =  0.985188842261
test count =  7044
SPAM is SPAM =  2933
SPAM is LEGIT =  118
LEGIT is SPAM =  2
LEGIT is LEGIT =  3991


In [ ]:
def getMeanClassifier(classifier1, classifier2):
    curClasses, curFrequencies = classifier1
    meanClasses, meanFrequencies = classifier2
    for clazz in curClasses:
        meanClasses[clazz] = (meanClasses[clazz] + curClasses[clazz]) / 2
    for clazz, word in curFrequencies:
        meanFrequencies[clazz, word] = (meanFrequencies[clazz, word] + curFrequencies[clazz, word]) / 2
    return meanClasses, meanFrequencies

In [20]:
TEST_DATA_FILE  = 'kaggle_test_data_tab.csv.gz'

with open('my_submission.csv' , 'w') as fout:
    writer = csv.writer(fout)
    writer.writerow(['Id','Prediction'])
    with gzip.open(TEST_DATA_FILE) if TEST_DATA_FILE.endswith('gz') else open(TEST_DATA_FILE) as input_file:            
        headers = input_file.readline()        
        for i, line in enumerate(input_file):
            trace(i)
            parts = line.decode('utf-8').strip().split('\t')
            url_id = int(parts[0])        
    #         url = parts[2]        
            html_data = base64.b64decode(parts[3])
            title, data, aa = html2word(html_data)
            writer.writerow([url_id, classify(curClassifier, data)])

01:46:54 INFO:Complete items 00000
01:47:28 INFO:Complete items 01000
01:47:56 INFO:Complete items 02000
01:48:24 INFO:Complete items 03000
01:48:49 INFO:Complete items 04000
01:49:13 INFO:Complete items 05000
01:49:41 INFO:Complete items 06000
01:50:06 INFO:Complete items 07000
01:50:33 INFO:Complete items 08000
01:51:00 INFO:Complete items 09000
01:51:56 INFO:Complete items 10000
01:52:27 INFO:Complete items 11000
01:52:57 INFO:Complete items 12000
01:53:23 INFO:Complete items 13000
01:53:55 INFO:Complete items 14000
01:54:26 INFO:Complete items 15000
01:54:59 INFO:Complete items 16000
